In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
train_df=pd.read_csv("D:\\machine learning projecsts folders\\hearth desese\\train.csv")
test_df=pd.read_csv("D:\\machine learning projecsts folders\\hearth desese\\test.csv")

In [ ]:
pd.set_option('display.max_columns',None)

In [ ]:
real_test_df=test_df.copy()

In [ ]:
train_df.drop(columns=['id'],inplace=True)
test_df.drop(columns=['id'],inplace=True)


In [ ]:
train_df['Heart Disease']=train_df['Heart Disease'].map({'Presence':1,'Absence':0})

In [ ]:
continues_value=[]
binary_number=[]
for i in train_df.columns.drop('Heart Disease'):
    if((train_df[i].nunique())<10):
        binary_number.append(i)
    else:
        continues_value.append(i)

In [ ]:
for i in binary_number:
    train_df[i]=train_df[i].astype("object")
    test_df[i]=test_df[i].astype('object')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
# from catboost import CatBoostClassifier
# 2. LightGBM
from lightgbm import LGBMClassifier
# 3. XGBoost
from xgboost import XGBClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder,PowerTransformer,RobustScaler,FunctionTransformer
from sklearn.metrics import confusion_matrix,classification_report,log_loss
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
X=train_df.drop(columns=['Heart Disease'])
y=train_df['Heart Disease']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.2)

In [ ]:
def heart_rate_features(X):
    X = X.copy()
    X['HR_Stress_Ratio'] = X['Max HR'] / (220 - X['Age'])
    X['High_ST_Depression'] = (X['ST depression'] > 4).astype(int)
    X['high risk']=((X['Age']*X['BP'])/X['Max HR'])
      # Based on your Cell 90 observation
    return X

In [ ]:

# 1. Logic from your EDA: Join Analysis with the Model
def add_eda_features(X):
    X = X.copy()
    # Stress Ratio based on your observation of Age vs Max HR
    X['HR_Stress_Ratio'] = X['Max HR'] / (220 - X['Age'])
    # High-risk marker from your EDA cell 90
    X['High_ST_Risk'] = (X['ST depression'] > 4).astype(int)
    return X

# 2. Advanced Preprocessing
# We use RobustScaler for BP because of the outliers you found
prepross = ColumnTransformer(transformers=[
    ('skewed', PowerTransformer(), ['Cholesterol', 'ST depression']),
    ('outliers', RobustScaler(), ['BP']),
    ('cats', OneHotEncoder(handle_unknown='ignore'), 
     ['Chest pain type', 'EKG results', 'Slope of ST', 'Thallium']),
    ('nums', StandardScaler(), ['Age', 'Max HR'])
], remainder='passthrough')

# 3. The Complete Pipeline
final_prep = Pipeline(steps=[
    ('engineering', FunctionTransformer(add_eda_features)),
    ('prep', prepross),
    ("model", XGBClassifier(objective='binary:logistic', eval_metric='logloss', random_state=42))
])

In [ ]:
# preprocessor=ColumnTransformer(transformers=[
#     ('skewed', PowerTransformer(), ['Cholesterol']),
#     ('outliers', RobustScaler(), ['BP', 'ST depression']),
#     ('cats', OneHotEncoder(handle_unknown='ignore'), 
#      ['Chest pain type', 'EKG results', 'Slope of ST', 'Thallium']),
#     ('nums', StandardScaler(), ['Age', 'Max HR'])
# ], remainder='passthrough')

In [ ]:
# prep=Pipeline(steps=[
#     ('custom_features', FunctionTransformer(heart_rate_features)),
#     ('prep', preprocessor),
#     ('model', XGBClassifier(n_estimators=500, learning_rate=0.01))
# ])

In [ ]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV

In [ ]:
param_grid = {
    'model__n_estimators': [200, 500],
    'model__max_depth': [3, 5, 7],
    'model__learning_rate': [0.01, 0.1],
    'model__subsample': [0.8, 1.0],
    'model__colsample_bytree': [0.8, 1.0],
    'model__gamma': [0, 0.1, 0.2],
   'model__reg_lambda': [1, 1.5, 2]
}
    

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(
    final_prep,
    param_grid,
    scoring='roc_auc',
    cv=cv,
    n_jobs=-1,
    verbose=2
)

grid.fit(X, y)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.","Pipeline(step...=None, ...))])"
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'model__colsample_bytree': [0.8, 1.0], 'model__gamma': [0, 0.1, ...], 'model__learning_rate': [0.01, 0.1], 'model__max_depth': [3, 5, ...], ...}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'roc_auc'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",StratifiedKFo... shuffle=True)
,"verbose verbose: intControls the verbosity: the higher, the more messag

In [ ]:
print("Best ROC-AUC:", grid.best_score_)
print("Best Params:", grid.best_params_)


Best ROC-AUC: 0.9553819259362534
Best Params: {'model__colsample_bytree': 0.8, 'model__gamma': 0, 'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 500, 'model__reg_lambda': 2, 'model__subsample': 1.0}


In [ ]:
best_model = grid.best_estimator_


In [ ]:
test_df.isna().sum()

Age                        0
Sex                        0
Chest pain type            0
BP                         0
Cholesterol                0
FBS over 120               0
EKG results                0
Max HR                     0
Exercise angina            0
ST depression              0
Slope of ST                0
Number of vessels fluro    0
Thallium                   0
dtype: int64

In [ ]:
output= best_model.predict(test_df)


In [ ]:
best_model.predict_proba(test_df)

array([[0.06523979, 0.9347602 ],
       [0.99186796, 0.00813202],
       [0.01236957, 0.9876304 ],
       ...,
       [0.95135385, 0.04864615],
       [0.7983745 , 0.20162553],
       [0.9738881 , 0.02611191]], shape=(270000, 2), dtype=float32)

In [ ]:
submission = pd.DataFrame({
    'id':real_test_df['id'],
    'Heart Disease': (output.ravel())
})
submission.to_csv("submission_fs5.csv", index=False)

submission.head()

,id,Heart Disease
0,630000,1
1,630001,0
2,630002,1
3,630003,0
4,630004,0


In [ ]:
round(0.9553984416952519-0.9553788567122112,3)

0.0

In [ ]:
# Check Feature Importance
grid.best_estimator_.named_steps['model'].feature_importances_
# Note: You'll need to map these back to your OneHotEncoded column names

array([5.19575842e-04, 1.50332442e-02, 2.88192590e-04, 1.65512692e-03,
       1.43308876e-04, 4.81767318e-04, 1.46196604e-01, 5.29117603e-03,
       9.16482895e-05, 1.09843491e-02, 7.46279284e-02, 6.26383757e-04,
       9.52452392e-05, 5.09527564e-01, 8.45816903e-05, 6.28323182e-02,
       3.38514196e-03, 6.90080551e-03, 3.08688879e-02, 1.07387306e-04,
       8.06795880e-02, 4.94388975e-02, 1.40306103e-04, 0.00000000e+00],
      dtype=float32)